In [7]:
import networkx as nx
import random
# import easygui as eui
import math
import time
import sys
import os
from termcolor import colored
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [8]:
CLASSES_PATH = os.path.dirname(os.path.abspath('../Classes/'))
if not (CLASSES_PATH in sys.path):
    sys.path.append(CLASSES_PATH)
from Classes.Files_Handler_Class import Files_Handler
from Classes.Resd_Network_Infos_Class import Resd_Network_Infos
from Classes.SIR_Diffusion_Model_Class import SIR_Diffusion_Model
from Classes.Get_Past_Results_Class import Get_Past_Results
from Classes.Network_Infos_Writer_Class import Network_Infos_Writer
from Classes.Memetic_Class import Memetic


In [9]:
seed_size_list = [[1, 2, 3, 4, 5, 6, 8, 10]]
datasets_list = ['04 - Irvine (1663 Node and 51 Layer)/Irvine.edgeslist']

root_file_path = 'D:/Masters thesis/Methods For Compare/Datasets/'


color_list = ["light_red", "light_green", "light_yellow",
                    "light_blue","light_magenta", "light_cyan",
                    "blue", "red", "white", "green", "yellow",
                        "magenta", "cyan", ]
tqdm_color_list = ['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'black', 'white']


files_handler_obj = Files_Handler()
SIR_diffusion_model_obj = SIR_Diffusion_Model()
resd_network_infos_obj = Resd_Network_Infos()
memetic_obj = Memetic()


source_code_path = str(os.getcwd())
source_code_path = source_code_path.replace("\\", "/")

draw_status = False

In [ ]:
for i, path in enumerate(datasets_list):
    file_path = root_file_path + path
    for item in seed_size_list[i]:
        print(path, item)
        seed_set_size = item
#--------------------------------------------------------------------------------------------------------------------------
        if file_path is None or file_path == '':
            sys.exit("File Selection Canceled !")
        file_info = files_handler_obj.get_file_path_info(file_path)
        network_name = file_info['name']
        network_type = file_info['type']
        network_path = file_info['path']
        if network_name == "":
            sys.exit("Dont Network Selection!")
        file_info

#--------------------------------------------------------------------------------------------------------------------------
        get_past_results_obj = Get_Past_Results(file_info['path'], file_info['name'])
        network_infos_writer_obj = Network_Infos_Writer(file_info['path'], file_info['name'])
#--------------------------------------------------------------------------------------------------------------------------
        (
            network_layers_info,
            network_layers_nodes,
            entra_layer_edges,
            entra_layer_edges_features,
            inter_layer_edge,
        ) = resd_network_infos_obj.read_nodeFrom_layerFrom_nodeTo_layerTo(
            network_path, network_name, network_type
        )
#--------------------------------------------------------------------------------------------------------------------------
        print(file_info['name'], '\n')
        sum_of_layers_nodes = 0
        network_layers_count = len(network_layers_info)
        graphs_of_network = [None] * network_layers_count
        network_entier_edges = ""
        layers_nodes_infect_scale = []

        i = 0
        j = 0
        while i < network_layers_count:
            graphs_of_network[i] = nx.Graph()
            network_layers_nodes[i] = list(set(network_layers_nodes[i]))
            layers_nodes_infect_scale.append({})
            if len(network_layers_nodes[i]) > 0:
                # graphs_of_network[i].add_nodes_from(network_layers_nodes[i])
                graphs_of_network[i].add_edges_from(entra_layer_edges[i])

                graphs_of_network[i].graph["id"] = i
                graphs_of_network[i].graph["node_num"] = graphs_of_network[i].number_of_nodes()
                graphs_of_network[i].graph["edge_num"] = graphs_of_network[i].number_of_edges()

                print(colored("Layer "  + str(i) + ": " + str(graphs_of_network[i].number_of_nodes()) + " Node And " +
                            str(graphs_of_network[i].number_of_edges()) + " Edge", color_list[j]))
                # print(colored(graphs_of_network[i].graph['k_shell_info'], color_list[i]))
            i += 1
            j += 1
            if j >= len(color_list):
                j = 0

        network_entier_nodes_list = []
        for item in network_layers_nodes:
            network_entier_nodes_list += item
            sum_of_layers_nodes += len(item)

        network_entier_nodes_list = list(set(network_entier_nodes_list))
        network_entier_nodes_count = len(network_entier_nodes_list)
        print()
        print("network entier nodes : " + colored(str(network_entier_nodes_count), "yellow"))
        MA_IMMULTI_path = files_handler_obj.make_dir(file_info['path'], 'MA_IMMULTI')
#--------------------------------------------------------------------------------------------------------------------------
        pop = 100
        maxgen = 50
        beta = 0.01
        seed_sets = []
        for m in range(math.floor(pop/3)):
            s_1 = random.sample(network_entier_nodes_list, seed_set_size)
            seed_sets.append(sorted(s_1, key=memetic_obj.str_to_int_func))
#--------------------------------------------------------------------------------------------------------------------------
        snen_by_synthetic_degree, synthetic_degree_mean, k_top_index  = memetic_obj.sort_networks_entier_nodes_by_synthetic_degree(graphs_of_network, network_entier_nodes_list, seed_set_size)
        k_top_synthetic_degree_list = snen_by_synthetic_degree[:(seed_set_size + 1)]
        bottom_synthetic_degree_list = snen_by_synthetic_degree[(seed_set_size + 1):]

#--------------------------------------------------------------------------------------------------------------------------
        for m in range(math.floor(pop/3)):
            s_2 = memetic_obj.select_k_nodes_by_synthetic_degree(k_top_synthetic_degree_list, bottom_synthetic_degree_list, seed_set_size)
            seed_sets.append(sorted(s_2, key=memetic_obj.str_to_int_func))
#--------------------------------------------------------------------------------------------------------------------------
        for m in range(math.ceil(pop/3)):
            s_3 = memetic_obj.select_k_nodes_by_highest_degree(k_top_synthetic_degree_list, bottom_synthetic_degree_list,
                                                        graphs_of_network, network_entier_nodes_list, seed_set_size)
            seed_sets.append(sorted(s_3, key=memetic_obj.str_to_int_func))
            
#--------------------------------------------------------------------------------------------------------------------------
        pc = 0.6
        new_pc = int(len(seed_sets) * pc)
        if new_pc % 2 != 0:
            new_pc += 1
        pm = 0.4
        pl = 0.5
        
#--------------------------------------------------------------------------------------------------------------------------
        local_search_candids = random.sample(seed_sets, int(len(seed_sets) * pl))
        for gen in local_search_candids:
            for item in gen:
                if not(item in dict(snen_by_synthetic_degree)):
                    print(item)

#--------------------------------------------------------------------------------------------------------------------------
        for g in range(maxgen):
            print(colored(f'Running generation {g+1}th ...', 'light_yellow'))
            
            print(colored('crossover ...', 'light_blue'))
            crossover_candids = random.sample(seed_sets, new_pc)
            pbar = tqdm(total=len(crossover_candids))
            pbar.unit = ' Gen'
            j = 1
            while j < len(crossover_candids):
                new_gen_a, new_gen_b = Memetic.crossover(crossover_candids[j-1], crossover_candids[j], network_entier_nodes_list)
                seed_sets.append(new_gen_a)
                seed_sets.append(new_gen_b)
                j += 2
                if j > len(crossover_candids):
                    break
                pbar.update(2)
            pbar.close()
            
            print()
            print(colored('mutation ...', 'light_red'))
            mutation_candids = random.sample(seed_sets, int(len(seed_sets) * pm))
            with tqdm(mutation_candids, unit=" Gen") as t_gen:
                    for gen in t_gen:
                        new_gen = Memetic.mutation(gen, snen_by_synthetic_degree,graphs_of_network, seed_set_size)
                        seed_sets.append(new_gen)
            
            print()
            print(colored('Local search ...', 'light_green'))
            local_search_candids = random.sample(seed_sets, int(len(seed_sets) * pl))
            with tqdm(local_search_candids, unit=" Gen") as t_gen:
                for gen in t_gen:
                    new_gen = Memetic.local_search_1(gen, network_entier_nodes_list, graphs_of_network, beta)
                    for item in new_gen:
                        if not(item in dict(snen_by_synthetic_degree)):
                            print(new_gen)
                    new_gen = Memetic.local_search_2(new_gen, network_entier_nodes_list, dict(snen_by_synthetic_degree), graphs_of_network)
                    seed_sets.append(new_gen)
            
            print()
            print(colored('spreading_approximation ...', 'light_yellow'))
            gens_spread_approximation = {}
            with tqdm(seed_sets, unit=" Gen") as t_gen:
                for j, gen in enumerate(t_gen):
                    gens_spread_approximation[j] = Memetic.spreading_approximation_multilayer(seed_sets[j],graphs_of_network, beta)

            gens_spread_approximation = dict(sorted(gens_spread_approximation.items(), key=lambda item: item[1], reverse=True))
            top_gens = []
            itr = iter(gens_spread_approximation.items())
            for j in range(pop):
                top_gens.append(seed_sets[next(itr)[0]])
            seed_sets = top_gens
            print('\n\n')

#--------------------------------------------------------------------------------------------------------------------------
        top_10_gens = seed_sets[:10]
#--------------------------------------------------------------------------------------------------------------------------
        print(f"Seed set size: {seed_set_size}")
        beta = 0.01
        landa = 0.7
        epoch = 10000
        SIR_diffusion_model_obj = SIR_Diffusion_Model()
        infection = SIR_diffusion_model_obj.synchronous_SIR_multilayer_with_seed_set_model(graphs_of_network, top_10_gens[0], beta, landa, epoch, network_entier_nodes_list)

        sir_results_infos = {}
        sir_results_infos['infection'] = infection
        sir_results_infos['percentage'] = infection / network_entier_nodes_count
        sir_results_infos['seed_set'] = top_10_gens[0]

        network_infos_writer_obj.write_results_in_file(MA_IMMULTI_path, f'infection k={seed_set_size} beta={beta} landa{landa} epoch{epoch}', sir_results_infos)
        print(f"Network entier nodes count: {network_entier_nodes_count}")
        print(f"Infected nodes count: {infection}")
        print(f"Percentage of infection: {sir_results_infos['percentage']}")

#--------------------------------------------------------------------------------------------------------------------------
        clear_output(wait=False)
#--------------------------------------------------------------------------------------------------------------------------
print('Finished!')



05 - Wainwright/Wainwright.edges 1
Wainwright 

Layer 1: 43 Node And 42 Edge
Layer 2: 121 Node And 124 Edge
Layer 3: 117 Node And 157 Edge
Layer 4: 44 Node And 48 Edge
Layer 5: 131 Node And 231 Edge
Layer 6: 55 Node And 101 Edge
Layer 7: 76 Node And 49 Edge
Layer 8: 107 Node And 182 Edge
Layer 9: 4 Node And 2 Edge
Layer 10: 120 Node And 162 Edge
Layer 11: 104 Node And 152 Edge
Layer 12: 57 Node And 46 Edge
Layer 13: 136 Node And 210 Edge
Layer 14: 21 Node And 15 Edge
Layer 15: 42 Node And 31 Edge
Layer 16: 73 Node And 69 Edge
Layer 17: 69 Node And 87 Edge
Layer 18: 12 Node And 6 Edge
Layer 19: 73 Node And 59 Edge
Layer 20: 2 Node And 1 Edge
Layer 21: 105 Node And 118 Edge
Layer 22: 73 Node And 66 Edge
Layer 23: 65 Node And 60 Edge
Layer 24: 25 Node And 14 Edge
Layer 25: 100 Node And 92 Edge
Layer 26: 8 Node And 4 Edge
Layer 27: 96 Node And 104 Edge
Layer 28: 67 Node And 64 Edge
Layer 29: 16 Node And 10 Edge
Layer 30: 79 Node And 72 Edge
Layer 31: 21 Node And 15 Edge
Layer 32: 42 Node A

 97%|█████████▋| 58/60 [00:00<00:00, 58142.84 Gen/s]



mutation ...


100%|██████████| 64/64 [00:00<00:00, 122.51 Gen/s]



Local search ...


...

 97%|█████████▋| 58/60 [00:00<00:00, 58746.59 Gen/s]



mutation ...


100%|██████████| 64/64 [00:00<00:00, 78.73 Gen/s] 



Local search ...


 67%|██████▋   | 75/112 [00:06<00:03, 11.27 Gen/s]